https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
import json
with open("docs.json", "r") as f:
    data = json.load(f)
len(data)

5198

In [11]:
import sqlite3

In [12]:
!cp ~/.mozilla/firefox/naoc2a85.default-release/places.sqlite ./places.sqlite

In [20]:
conn = sqlite3.connect('places.sqlite')

In [21]:
cur = conn.cursor()
res = cur.execute("""
    SELECT url
    FROM moz_bookmarks as mb
    LEFT JOIN moz_places as mp ON mb.fk = mp.id
    WHERE type = 1
    and url is not null
    and url not like 'javascript:%'
    and url not like 'place:%'
    and url not like 'view-source:%'
    and url not like 'about:%'
    and url not like 'http%.lan/%'
    and url not like 'http%.lan:%'
	and url not like 'http%.local:%'
	and url not like 'http%.local/%'
    GROUP BY url
""")

In [22]:
urls = set()
for (b_url,) in res:
    urls.add(b_url)

In [23]:
cur.close()
conn.close()
len(urls)

5831

In [7]:
from document_loaders import ThreadedUnstructuredURLLoader

In [8]:
u_loader = ThreadedUnstructuredURLLoader(urls=list(urls)[-100:], max_workers=100, mode="single", strategy="fast", show_progress_bar=False)

In [10]:
docs = u_loader.load()

ERROR:root:HTTPSConnectionPool(host='docs.apitools.com', port=443): Max retries exceeded with url: /docs/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0e971fb590>: Failed to establish a new connection: [Errno -2] Name or service not known'))
ERROR:root:HTTPConnectionPool(host='www.mashape.com', port=80): Max retries exceeded with url: /apis/musiXmatch (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0e9848a010>: Failed to establish a new connection: [Errno -2] Name or service not known'))


ERROR:root:'utf-8' codec can't decode byte 0xe0 in position 12937: invalid continuation byte


ERROR:root:HTTPConnectionPool(host='silex.sensiolabs.org', port=80): Max retries exceeded with url: /doc/organizing_controllers.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0f2c34fdd0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
ERROR:root:HTTPConnectionPool(host='31.194.38.115', port=8080): Max retries exceeded with url: /Gatweb/pgms/login/login.asp (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0fa4152a90>: Failed to establish a new connection: [Errno 113] No route to host'))


ERROR:root:HTTPSConnectionPool(host='backtest-rookies.com', port=443): Max retries exceeded with url: /2018/02/09/tradingview-pinescript-debugging-plotting-tips-tricks/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0fa4193690>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


ERROR:root:HTTPConnectionPool(host='www.voidspace.org.uk', port=80): Max retries exceeded with url: /python/articles/authentication.shtml (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0e928c5590>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


ERROR:root:HTTPSConnectionPool(host='rexray.io', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f0e96ae1390>, 'Connection to rexray.io timed out. (connect timeout=None)'))


In [19]:
dict(docs[0])

{'page_content': 'Not Found\nThe requested URL was not found on this server.',
 'metadata': {'source': 'http://labs.alcacoop.it/doku.php?id=projects:openwrt:config'}}

In [20]:
import json
import config
with open(config.DATA_DIR / 'bookmarks.json', "w") as f:
    json.dump([dict(doc) for doc in docs], f)

In [1]:
import json
with open(config.DATA_DIR / 'bookmarks.json', "r") as f:
    data = json.load(f)

In [2]:
data[0]

{'page_content': 'ZBar bar code reader ZBar bar code reader home about download support iPhone wiki community Download Now! symbologies The flexible, layered implementation facilitates bar code scanning and decoding for any application: use it stand-alone with the included GUI and command line programs, easily integrate a bar code scanning widget into your Qt, GTK+ or PyGTK GUI application, leverage one of the script or programming interfaces (Python, Perl, C++) ...all the way down to a streamlined C library suitable for embedded use. GNU LGPL 2.1 Applications Retail Automated document processing Inventory tracking Mobile applications Features ® real time small memory footprint small code size - the core scanner and EAN decoder represent under 1K lines of C code not limited to images no floating point operations suitable for embedded applications using inexpensive processors/hardware modular components can be used together or separately spadix@users.sourceforge.net Copyright 2007-2010 

In [3]:
from langchain.schema import Document
restored = [Document(**doc) for doc in data]

In [4]:
len(restored)

184

In [13]:
from unstructured.partition.html import partition_html

In [14]:
headers = { 
    "User-Agent": "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/113.0.5672.127 Safari/537.36" 
}

In [42]:
elements = partition_html(url=list(urls)[1], headers=headers)
from unstructured.partition.auto import partition
elements = partition(url=list(urls)[1], headers=headers)

In [43]:
elements[0].metadata.to_dict()

{'filetype': 'text/html',
 'page_number': 1,
 'url': 'http://stackoverflow.com/questions/27764020/different-between-phonegap-and-intel-xdk-or-others',
 'links': [{'text': '\r\n\t\t\t\t\t', 'url': 'https://stackoverflow.com'}]}

In [44]:
text = "\n\n".join([str(el) for el in elements])
text

'Stack Overflow\n\nAbout\n\nProducts\n\nFor Teams\n\nStack Overflow\r\n\t\t\t\t\t\t\tPublic questions & answers\n\nStack Overflow for Teams\r\n\t\t\t\t\t\t\tWhere developers & technologists share private knowledge with coworkers\n\nTalent\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\tBuild your employer brand\n\nAdvertising\r\n\t\t\t\t\t\t\tReach developers & technologists worldwide\n\nLabs\r\n\t\t\t\t\t\t\tThe future of collective knowledge sharing\n\nAbout the company\n\nLoading…\n\ncurrent community\r\n            \r\n            \r\n                \r\n                    \r\n                        \r\n                    \r\n                \r\n            \r\n        \r\n        \r\n            \r\n                    \r\n                            \r\n                \r\n        \r\n        \r\n            Stack Overflow\r\n        \r\n    \r\n\r\n    \r\n    \r\n            help\r\n            chat\r\n    \r\n\r\n                    \r\n                    \r\n                        

In [41]:

import re
text = text.replace('\r\n', '\n').replace('\t', ' ')
text = re.sub(r'[ ]+', ' ', text).replace('\n ', '\n')
text = re.sub(r'\n+', '\n', text)
text

'Stack Overflow\nAbout\nProducts\nFor Teams\nStack Overflow\nPublic questions & answers\nStack Overflow for Teams\nWhere developers & technologists share private knowledge with coworkers\nTalent\nBuild your employer brand\nAdvertising\nReach developers & technologists worldwide\nLabs\nThe future of collective knowledge sharing\nAbout the company\nLoading…\ncurrent community\nStack Overflow\nhelp\nchat\nMeta Stack Overflow\nyour communities \nSign up or log in to customize your list. \nmore stack exchange communities\ncompany blog\nLog in\nSign up\nNew! OverflowAI: Where Community & AI Come Together\nHome\nPublic\nQuestions\nTags\nUsers\nCompanies\nCollectives\nExplore Collectives\nTeams\nStack Overflow for Teams\n– Start collaborating and sharing organizational knowledge.\nCreate a free Team\nWhy Teams?\nTeams\nCreate free Team\nCollectives™ on Stack Overflow\nFind centralized, trusted content and collaborate around the technologies you use most.\nLearn more about Collectives\nTeams\nQ

In [17]:
!playwright install
from langchain.document_loaders import PlaywrightURLLoader

BEWARE: your OS is not officially supported by Playwright; downloading fallback build.
146.4 Mb [                    ] 0% 0.0s146.4 Mb [                    ] 0% 42.4s146.4 Mb [                    ] 0% 41.1s146.4 Mb [                    ] 0% 34.8s146.4 Mb [                    ] 0% 31.2s146.4 Mb [                    ] 0% 33.5s146.4 Mb [                    ] 0% 30.8s146.4 Mb [                    ] 0% 27.8s146.4 Mb [                    ] 0% 26.7s146.4 Mb [                    ] 0% 25.5s146.4 Mb [                    ] 0% 27.1s146.4 Mb [                    ] 0% 23.9s146.4 Mb [                    ] 0% 23.4s146.4 Mb [                    ] 1% 23.0s146.4 Mb [                    ] 1% 22.7s146.4 Mb [                    ] 1% 22.2s146.4 Mb [                    ] 1% 22.0s146.4 Mb [                    ] 1% 21.7s146.4 Mb [                    ] 1% 21.5s146.4 Mb [                    ] 1% 21.4s146.4 Mb [                    ] 1% 21.6s146.4 Mb [                    ] 1% 21.1s146.4 Mb [                    ] 1%

In [23]:
p_loader = PlaywrightURLLoader(urls=list(urls)[-50:], remove_selectors=["header", "footer"])

In [24]:
await p_loader.aload()

Error fetching or processing https://petals.ml/, exception: net::ERR_NAME_NOT_RESOLVED at https://petals.ml/
=========================== logs ===========================
navigating to "https://petals.ml/", waiting until "load"
Error fetching or processing http://qnap.h.loopback.it:5050/, exception: net::ERR_CONNECTION_REFUSED at http://qnap.h.loopback.it:5050/
=========================== logs ===========================
navigating to "http://qnap.h.loopback.it:5050/", waiting until "load"
Error fetching or processing https://www.xenialab.it/meo/web/index5.htm#my, exception: net::ERR_CERT_COMMON_NAME_INVALID at https://www.xenialab.it/meo/web/index5.htm#my
=========================== logs ===========================
navigating to "https://www.xenialab.it/meo/web/index5.htm#my", waiting until "load"
Error fetching or processing http://www.exploitsearch.net/, exception: net::ERR_NAME_NOT_RESOLVED at http://www.exploitsearch.net/
=========================== logs ===========================

[Document(page_content='404: NOT_FOUND\nCode: NOT_FOUND\nID: fra1::bp66h-1690673450555-70e3a59608bf\n\nRead our documentation to learn more about this error.', metadata={'source': 'http://themespectre.com/linen/'}),
 Document(page_content="Are you Human?\n\nWe couldn't verify your identity, please solve the captcha below.\n\nIf you feel you received this page in error, please contact w1zzard@techpowerup.com.", metadata={'source': 'https://www.techpowerup.com/reviewdb/PSUs/'}),
 Document(page_content='This repository has been archived by the owner on Jan 20, 2020. It is now read-only.\n\ncoinbase\n\ncoinbase-pro-trading-toolkit\n\nPublic archive\n\nNotifications\n\nFork\n    253\n\nStar\n          850\n\nDEPRECATED — The Coinbase Pro trading toolkit\n\nLicense\n\nApache-2.0 license\n\n850\n          stars\n\n253\n          forks\n\nActivity\n\nStar\n\nNotifications\n\nCode\n\nIssues\n          0\n\nPull requests\n          0\n\nActions\n\nSecurity\n\nInsights\n\nMore\n\nCode\n\nIssues\n